# [RAG 문제2]  
- 파인콘 벡터 DB에 저장
- LLM 질문 - > 답변  
- 전세사기피해에 관한 법률 질문만 받도록.  
- 이외의 질문은 '답변을 할 수 없습니다.'라고 답변.  
- 이미지는 어떻게 읽어올지 고민.  

In [30]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader
from langchain.schema import Document

loader = Docx2txtLoader("law_2.docx")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, 
    chunk_overlap=200,
)

data_list = loader.load_and_split(text_splitter=text_splitter)

## 이미지에 있는 수식을 수동으로 지정해서 data_list에 추가해줌. 

formula_1 = """
제3조(국세의 안분 방법 및 신청 등) 제1항 제1호 가목

가. 다음의 계산식에 따라 계산한 금액:

A × B ÷ C

A: 전세사기피해주택의 임대인이 세납한 고지 또는 신고 건별 국세 금액  
B: 전세사기피해주택의 가격  
C: 전세사기피해주택의 임대인이 보유한 모든 주택의 가격 합계액
"""

doc_formula_1 = Document(page_content=formula_1, metadata={"source": "수식 이미지 1"})

formula_2 = """
제4조(지방세의 안분 방법 및 신청 등) 제1항 제2호

2. 제1호 외의 지방세의 경우: 고지 또는 신고 건별로 각각 다음의 계산식에 따라 계산한 금액:

A × B ÷ C

A: 전세사기피해주택의 임대인이 세납한 고지 또는 신고 건별 지방세 금액  
B: 전세사기피해주택의 가격  
C: 전세사기피해주택의 임대인이 보유한 모든 주택의 가격 합계액
"""

doc_formula_2 = Document(page_content=formula_2, metadata={"source": "수식 이미지 2"})

data_list.extend([doc_formula_1, doc_formula_2])

load_dotenv()

embedding = OpenAIEmbeddings(model="text-embedding-3-large")

api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)

# database = PineconeVectorStore.from_documents(
#     documents=data_list,
#     embedding=embedding,
#     index_name='law-2-index',
# )

database = PineconeVectorStore(
    index_name='law-2-index',
    embedding=embedding,
)


query = "전세사기피해자들의 지방세 안분 관련 계산식이 어떻게 되나요?"

retrieved_docs = database.similarity_search(query=query, k=2)

prompt = '''
[identity]
-당신은 전세사기피해 관련 법률 전문가입니다.
-[context]를 참고해 사용자의 질문에 답변하세요. 
-관련 없는 질문엔 "관련 내용만 질문해주세요."라고 대답하세요. 

[context]
{retrieved_docs}

[question]
{query}

'''


llm = ChatOpenAI(model_name='gpt-4.1-nano')

aimessage = llm.invoke(
    prompt.format(
        retrieved_docs=retrieved_docs,
        query=query
    )
)

aimessage.content

'전세사기피해자들의 지방세 안분 관련 계산식은 다음과 같습니다.\n\n1. 전세사기피해주택의 임대인이 신고 또는 고지받은 지방세 금액에 대해:\n   \n   A × B ÷ C\n\n   - A: 전세사기피해주택 임대인이 납부한 해당 지방세 (고지 또는 신고 건별 금액)\n   - B: 전세사기피해주택의 시가표준액 또는 산정가액\n   - C: 임대인이 보유한 모든 주택의 시가표준액 또는 산정가액의 합계\n\n2. 계산 방법에는 세법에서 정하는 기준에 따라, 예를 들어 상속세, 증여세, 종합부동산세 등은 각각의 큰 금액 또는 계산식을 적용하며, \n\n3. 기타 국세는 별도 계산식을 따릅니다.\n\n이 계산식을 통해 각 피해자가 부담해야 할 지방세액이 산출됩니다.'